In [1]:
!pip install tensorflow-datasets

In [2]:
!python -m pip install --upgrade pip

Requirement already up-to-date: pip in c:\users\affinity\anaconda3\lib\site-packages (19.3.1)


In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.models import load_model
from datetime import datetime
#tf.enable_eager_execution()

Using TensorFlow backend.


In [4]:
train_data = tfds.load(name='cifar10:3.*.*', split='train[:80%]', as_supervised=True)
valid_data = tfds.load(name='cifar10:3.*.*', split='train[80%:]', as_supervised=True)
test_data = tfds.load(name='cifar10:3.*.*', split='test', as_supervised=True)

In [5]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    image = tf.image.random_contrast(image, lower=2.0, upper=2.0)
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_contrast(image, lower=2.0, upper = 2.0)
    image = tf.image.random_hue(image, max_delta = 0.8)
    image = tf.image.random_saturation(image, lower=2.0, upper=2.0)
    return image, label

In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same', input_shape=(32, 32, 3)),
    tf.keras.layers.BatchNormalization(),  # Gradient Vanishing / Gradient Exploding 이 일어나지 않도록 하는 아이디어 중의 하나이다
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='softmax')
    
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 128)       3584      
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 128)       512       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 256)       295168    
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 256)       590080    
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 256)       1024      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 256)         0

In [7]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [8]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

train_data = train_data.map(scale).shuffle(50000).repeat(4).batch(64) 
valid_data = valid_data.map(scale).batch(64)
test_data = test_data.map(scale).batch(64)

In [9]:
dt = datetime.now().strftime('%Y%m%d-%H%M%S')

cp_callback = tf.keras.callbacks.ModelCheckpoint(   
    'checkpoints/{}.hdf5'.format(dt),
    monitor='val_accuracy',
    save_best_only=True)

In [10]:
class MyCallback(tf.keras.callbacks.Callback):

    def on_epoch_start(self, epoch, logs=None):
        print('Epoch {} begins at {}: {}'.format(epoch, datetime.now().time(), logs))

    def on_epoch_end(self, epoch, logs=None):
        print('Epoch {} ends at {}: {}'.format(epoch, datetime.now().time(), logs))

In [11]:
history = model.fit(train_data,
                    epochs=20,
                    validation_data=valid_data,
                    verbose=0,
                    callbacks=[cp_callback, MyCallback()])

Epoch 0 ends at 10:12:02.778439: {'loss': 1.7848540184497834, 'accuracy': 0.45964375, 'val_loss': 1.199775561405595, 'val_accuracy': 0.6008}
Epoch 1 ends at 10:16:04.590190: {'loss': 0.8971222634077072, 'accuracy': 0.704375, 'val_loss': 1.0797167207784713, 'val_accuracy': 0.6151}
Epoch 2 ends at 10:20:11.827649: {'loss': 0.543234035307169, 'accuracy': 0.8132125, 'val_loss': 0.8240562557794486, 'val_accuracy': 0.7276}
Epoch 3 ends at 10:24:26.830897: {'loss': 0.36892248053252696, 'accuracy': 0.87275624, 'val_loss': 0.6473339634716131, 'val_accuracy': 0.7969}
Epoch 4 ends at 10:28:44.768282: {'loss': 0.25954890947192905, 'accuracy': 0.90995, 'val_loss': 0.5959679695071688, 'val_accuracy': 0.818}
Epoch 5 ends at 10:33:07.977395: {'loss': 0.18837507278323173, 'accuracy': 0.9352875, 'val_loss': 0.6253098944190202, 'val_accuracy': 0.8217}
Epoch 6 ends at 10:37:37.075387: {'loss': 0.14560401912592352, 'accuracy': 0.950725, 'val_loss': 0.747721675000373, 'val_accuracy': 0.8153}
Epoch 7 ends at

In [12]:
model.evaluate(test_data)

157/157 [==============================]0.6674 - accuracy: 0.859 - 0s 92ms/step - loss: 0.8031 - accuracy: 0.843 - 0s 81ms/step - loss: 0.7321 - accuracy: 0.84 - 0s 78ms/step - loss: 0.7533 - accuracy: 0.85 - 0s 75ms/step - loss: 0.7289 - accuracy: 0.86 - 0s 72ms/step - loss: 0.6692 - accuracy: 0.87 - 0s 69ms/step - loss: 0.6718 - accuracy: 0.86 - 1s 67ms/step - loss: 0.7493 - accuracy: 0.85 - 1s 65ms/step - loss: 0.7167 - accuracy: 0.86 - 1s 63ms/step - loss: 0.7131 - accuracy: 0.86 - 1s 62ms/step - loss: 0.6724 - accuracy: 0.86 - 1s 61ms/step - loss: 0.6713 - accuracy: 0.86 - 1s 61ms/step - loss: 0.6707 - accuracy: 0.86 - 1s 61ms/step - loss: 0.6663 - accuracy: 0.86 - 1s 61ms/step - loss: 0.6895 - accuracy: 0.85 - 1s 60ms/step - loss: 0.7143 - accuracy: 0.85 - 1s 61ms/step - loss: 0.7189 - accuracy: 0.85 - 1s 60ms/step - loss: 0.7435 - accuracy: 0.84 - 1s 60ms/step - loss: 0.7456 - accuracy: 0.84 - 1s 60ms/step - loss: 0.7416 - accuracy: 0.84 - 1s 60ms/step - loss: 0.7122 - accuracy:

[0.8324826537234009, 0.8403]

In [13]:
m = tf.keras.models.load_model('checkpoints/{}.hdf5'.format(dt))